In [47]:
from bs4 import BeautifulSoup
import requests
import re
import time

In [48]:
base_url = 'https://namu.wiki'

In [49]:
list_url = '/w/%EC%9D%BC%EB%B3%B8%20%EC%95%A0%EB%8B%88%EB%A9%94%EC%9D%B4%EC%85%98/%EB%AA%A9%EB%A1%9D'

In [54]:
def get_soup(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def get_table(soup):
    try:
        table = soup.find(string="장르").find_parent('table', class_='wiki-table')
        return table
    except AttributeError:
        return None

def get_image(soup):
    table = get_table(soup)
    if table:
        img = table.select_one('.wiki-image').get('data-src')
        return img
    else:
        try:
            img = soup.select_one('.wiki-inner-content .wiki-image').get('href')
            return img
        except AttributeError:
            return None

def get_genre(soup):
    table = get_table(soup)
    if table:
        genre = table.find(string="장르").find_parent('tr').find_all('td')[1].get_text()
        return genre
    
def get_anime(anime_list_links):
    for anime_link in anime_list_links:
        title = anime_link.get('title')
        soup = get_soup(base_url + anime_link.get('href'))
        
        genre = get_genre(soup)
        image_link = get_image(soup)
        time.sleep(1)
        
        data = {
            'title': title,
            'genre': genre,
            'image_link': image_link
        }
        print(data)
        
def get_animes_by_content(anime_content_links):
    for anime_content_link in anime_content_links:
        anime_list_links = anime_content_link.select('ul > li > .wiki-paragraph > .wiki-link-internal')
        datas = get_anime(anime_list_links)
        print(datas)
        break

def get_animes_by_consonant(anime_consonant_links):
    for anime_consonant_link in anime_consonant_links:
        soup = get_soup(base_url + anime_consonant_link.get('href'))
        anime_content_links = soup.select('.wiki-inner-content > .wiki-heading-content')
        get_animes_by_content(anime_content_links)

def get_animes():
    soup = get_soup(base_url + list_url)
    anime_consonant_links = soup.select('.wiki-heading-content > .wiki-paragraph > .wiki-link-internal')[1::2]
    
    get_animes_by_consonant(anime_consonant_links)

In [55]:
get_animes()

{'title': '가난뱅이 신이!', 'genre': '개그', 'image_link': '//w.namu.la/s/8911f358b21b7a3fdefa677adb9a221b7005bcdb9d597464302f8c5d53458ad0eeb9f278f3efb18f1280b7f74ed3da9fb6a2ba73274980846d4c6d10a617fc11ce4916bb0cb84c4ce6e512b2f7d79ba47bcf873388d763e52509f565a8bc3350'}
{'title': '가르쳐줘! 갸루코쨩', 'genre': '학원, 코미디, 일상', 'image_link': '//w.namu.la/s/ce4dc0687d9a2160dedaa0a8d4eef698806c706cc4a8c07144456936715a46047a0221fd7877a25694e1f3ff4a380190c4d60cc039404ac48b3a3c5ac4744e5214114c4a29468928afed0dae2f81eadcca0a7ccc259ca7df36e79904aacfc333'}
{'title': '가면의 메이드가이', 'genre': None, 'image_link': None}
{'title': '가부키부!', 'genre': '가부키, 학원물', 'image_link': '//w.namu.la/s/9605dc3106a19cc594f2123f94938c30afba87e39f74ecb07c0735c5ab776ec73fdc35fd3e700c0186bf69dfdaf3d90478c6ce6a7860beefe31e108b1a95ecd49d3832e75a3e14b57a424070078cea5a56cf4b584a9ce68ab3c23e27c8e2cacc'}
{'title': '가브릴 드롭아웃', 'genre': '판타지, 일상, 코미디', 'image_link': '//w.namu.la/s/c9df8fc9b76441081faa72d2088eb1e644d63f83deee1386138d89eb30fd914a00e74

KeyboardInterrupt: 